In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif

from imblearn.over_sampling import SMOTE

In [5]:
str = 'event_7-abortion_finger'

### Preprocessing

In [6]:
# input data
Data = pd.read_csv(str + '.csv')
print(Data.shape)

# imbalance
print('number of active: ', sum(Data.iloc[:,-1] != 0))
print('number of inactive: ', sum(Data.iloc[:,-1] == 0))


(1647, 883)
number of active:  59
number of inactive:  1588


In [ ]:
Columns = []

for i in range(Data.shape[1]):
    for j in range(Data.shape[0]):
        a = Data.iloc[j,i]
        if a == '#NAME?':
            Columns.append(i)
            #print(i)
            break
            
list1 = list(Data.columns)

for i in range(len(Columns)):
    Data.drop(list1[Columns[i]],axis = 1,inplace = True)

X = Data.iloc[:,1:-1]
y = Data.iloc[:,-1]

print(X.shape)
print(y.shape)
print(y.unique())

(1647, 881)
(1647,)
[0 1]


In [ ]:
Columns_ = []

for i in range(X.shape[1]):
    if X.iloc[:,i].max() >= 10000:
        Columns_.append(i)

print(Columns_)

list2 = list(X.columns)

for i in range(len(Columns_)):
    X.drop(list2[Columns_[i]],axis = 1,inplace = True)

X.shape

[]


(1647, 881)

In [ ]:
Columns_1 = []

for i in range(X.shape[1]):
    for j in range(X.shape[0]):
        a = X.iloc[j,i]
        if pd.isnull(a) == True:
            Columns_1.append(i)
            break
    
print(Columns_1)

list3 = list(X.columns)

for i in range(len(Columns_1)):
    X.drop(list3[Columns_1[i]],axis = 1,inplace = True)

X.shape

[]


(1647, 881)

In [ ]:
selector = VarianceThreshold()
selector = selector.fit(X)
X_var0 = selector.transform(X)
X_var0.shape

(1647, 679)

In [11]:
list4 = selector.get_feature_names_out()
X_var0 = pd.DataFrame(X_var0)
X_var0.columns = list4
X_var0.shape

(1647, 679)

In [ ]:
result = mutual_info_classif(X_var0,y)
k = result.shape[0] - sum(result <= 0)
mlc = SelectKBest(mutual_info_classif, k=k).fit(X_var0, y)
X_fsmic = mlc.transform(X_var0)
cr = cross_val_score(RandomForestClassifier(n_estimators=10,random_state=0),X_fsmic,y,cv=5).mean()

print(k) 
print(cr)
print(X_fsmic.shape)

370
0.9575002302661877
(1647, 370)


In [13]:
list5 = mlc.get_feature_names_out()
X_fsmic = pd.DataFrame(X_fsmic)
X_fsmic.columns = list5
X_fsmic.shape

(1647, 370)

In [14]:
X_fsmic.to_csv(str + '-2.csv')
X_ = pd.read_csv(str + '-2.csv', index_col=0)
list_ = list(X_.columns)

y.to_csv(str + '-3.csv')
y_ = pd.read_csv(str + '-3.csv', index_col=0)
y_ = np.array(y_).reshape(-1)

X_.shape, y_.shape

((1647, 370), (1647,))

In [ ]:
# preprocessing.StandardScaler
# training set
scaler = StandardScaler()
scaler =scaler.fit(X_)
X_std = scaler.transform(X_)
X_std = pd.DataFrame(X_std)
X_std.columns = X_.columns

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_std,y_,test_size=0.2,random_state=0)

Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((1317, 370), (330, 370), (1317,), (330,))

In [ ]:
Ytrain=pd.Series(Ytrain)
Ytest=pd.Series(Ytest)
print(Ytrain.value_counts())
print(Ytest.value_counts())

0    1268
1      49
Name: count, dtype: int64
0    320
1     10
Name: count, dtype: int64


In [ ]:
sm = SMOTE(random_state=42)
Xtrain_,Ytrain_ = sm.fit_resample(Xtrain,Ytrain)

In [18]:
n_sample_ = Xtrain_.shape[0]
n_sample_

2536

In [19]:
print(Ytrain_.value_counts())

0    1268
1    1268
Name: count, dtype: int64


In [20]:
Xtrain.to_csv(str + '-Xtrain.csv')
Xtrain_.to_csv(str + '-Xtrain_.csv')
Xtest.to_csv(str + '-Xtest.csv')
Ytrain.to_csv(str + '-Ytrain.csv')
Ytrain_.to_csv(str + '-Ytrain_.csv')
Ytest.to_csv(str + '-Ytest.csv')